# Load Data and Merge into Proper DF to Save as CSV file

In [1]:
import os
os.chdir('../saliency_model/')

import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib

import pandas as pd
import seaborn as sns
from glob import glob

matplotlib.rcParams.update({'font.size': 15})

### Pick Metric
NSS, SIM, IG, AUC

In [2]:
plot_metric = 'IG'

In [3]:
# pick directory
save_dir ='../report/plots/'

In [4]:
os.chdir('../results/')
glob('*.p')

['19-02-01_results_over_baseline_faces_07.p',
 '19-02-01_results_plain_models.p',
 '19-02-01_results_face_07.p',
 '2019-05-12_results-with-std.p',
 '19-02-01_results_over_baseline_faces_05.p',
 '2019-05-11_results-with-std.p',
 '19-02-01_results_over_baseline_faces_03.p',
 '19-01-31_results.p',
 '19-02-01_results_face_03.p',
 '19-02-01_results_face_05.p']

## Pick the Dataset and Load Required Data

In [5]:
(summary, context, model, metric) = pickle.load(open( '19-02-01_results_plain_models.p', "rb"))

In [6]:
print(np.shape(summary))
print(context)
print(model)
print(metric)

(6, 9, 4)
['all', 'person', 'no person', 'animal', 'electronic', 'sports']
['aim', 'cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun', 'dg', 'faces']
['nss', 'sim', 'ig', 'auc']


In [7]:
(summary_f03, context_f03, model_f03, metric_f03) = pickle.load(open( '19-02-01_results_face_03.p', "rb"))
(summary_f05, context_f05, model_f05, metric_f05) = pickle.load(open( '19-02-01_results_face_05.p', "rb"))
(summary_f07, context_f07, model_f07, metric_f07) = pickle.load(open( '19-02-01_results_face_07.p', "rb"))

#### Vizualize the inside of one of the models

In [8]:
model_f03

['aim_face_03',
 'cas_face_03',
 'covsal_face_03',
 'gbvs_face_03',
 'icf_face_03',
 'ik_face_03',
 'sun_face_03']

## Data Loading

### Get ID of the Metric

In [9]:
if plot_metric == 'NSS':
    metric_id = 0
elif plot_metric == 'SIM':
    metric_id = 1
elif plot_metric == 'IG':
    metric_id = 2
elif plot_metric == 'AUC':
    metric_id = 3
else:
    raise ValueError('Wrong metric name provided.')
    
metric_id

2

### Load the scores

In [10]:
scores = summary[:,:,metric_id]
f03_scores = summary_f03[:,:,metric_id]
f05_scores = summary_f05[:,:,metric_id]
f07_scores = summary_f07[:,:,metric_id]

## Change scores into DataFrames
### No Faces Data Frame

In [11]:
df = pd.DataFrame(data=scores, columns=model, index=context)
df.columns.name='models'
df.index.name='context'

df = df.transpose()
df

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,3.601486,3.507835,3.753299,3.815986,4.324051,2.754290
cas,4.042589,3.957036,4.181276,4.193363,4.651863,3.887734
covsal,2.988373,2.990500,2.984926,2.751369,3.440113,2.878326
gbvs,4.389129,4.261897,4.595379,4.601708,4.949470,4.191043
icf,3.967415,3.761353,4.301451,4.290669,4.631382,3.111732
ik,3.782932,3.630731,4.029657,3.795496,4.468836,3.324104
sun,3.689604,3.501879,3.993918,3.997621,4.343237,3.224621
dg,3.110164,2.745240,3.701723,2.988734,3.812890,1.685864
faces,-17.790197,-17.857992,-11.146258,-19.223443,-15.835044,-21.367014


#### Separate Faces From the rest of the results

In [12]:
faces = df.loc['faces']
df = df.loc[['aim','cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun', 'dg']]

#### Get Mean Values across Contexts and Models

In [13]:
df['mean context'] = df.mean(axis=1)
df = df.transpose()
df['mean model'] = df.mean(axis=1)
df = df.transpose()
df

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,3.601486,3.507835,3.753299,3.815986,4.324051,2.754290,3.626158
cas,4.042589,3.957036,4.181276,4.193363,4.651863,3.887734,4.152310
covsal,2.988373,2.990500,2.984926,2.751369,3.440113,2.878326,3.005601
gbvs,4.389129,4.261897,4.595379,4.601708,4.949470,4.191043,4.498104
icf,3.967415,3.761353,4.301451,4.290669,4.631382,3.111732,4.010667
ik,3.782932,3.630731,4.029657,3.795496,4.468836,3.324104,3.838626
sun,3.689604,3.501879,3.993918,3.997621,4.343237,3.224621,3.791813
dg,3.110164,2.745240,3.701723,2.988734,3.812890,1.685864,3.007436
mean model,3.696461,3.544559,3.942704,3.804368,4.327730,3.132214,3.741339


In [14]:
df_mean_model = df.loc['mean model']
df_mean_context = df.transpose().loc['mean context']

### Faces with $\alpha$ = 0.3

In [15]:
df_f03 = pd.DataFrame(data=f03_scores, columns=model_f03, index=context_f03)
df_f03.columns.name='models'
df_f03.index.name='context'

# Rename columns of the face plot
col = {'aim_face_03':'aim', 'cas_face_03':'cas', 'covsal_face_03':'covsal', 'gbvs_face_03':'gbvs',
       'icf_face_03':'icf', 'ik_10_face_03':'ik_10', 'sun_face_03':'sun', 'ik_face_03':'ik'}
df_f03 = df_f03.rename(index=str, columns=col).transpose()

df_f03

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,3.618003,3.527088,3.765382,3.845403,4.344158,2.759067
cas,4.058921,3.978041,4.190726,4.199506,4.688515,3.894333
covsal,3.040721,3.051236,3.023675,2.820840,3.496465,2.887945
gbvs,4.401902,4.283458,4.593905,4.607203,4.971440,4.194267
icf,3.973449,3.769190,4.304565,4.300628,4.643860,3.101522
ik,3.803815,3.665109,4.028663,3.802530,4.499704,3.337869
sun,3.722750,3.554025,3.996262,4.011587,4.393395,3.250412


#### Get Mean Values across Contexts and Models

In [16]:
df_f03['mean context'] = df_f03.mean(axis=1)
df_f03 = df_f03.transpose()
df_f03['mean model'] = df_f03.mean(axis=1)
df_f03 = df_f03.transpose()
df_f03

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,3.618003,3.527088,3.765382,3.845403,4.344158,2.759067,3.643184
cas,4.058921,3.978041,4.190726,4.199506,4.688515,3.894333,4.168340
covsal,3.040721,3.051236,3.023675,2.820840,3.496465,2.887945,3.053480
gbvs,4.401902,4.283458,4.593905,4.607203,4.971440,4.194267,4.508696
icf,3.973449,3.769190,4.304565,4.300628,4.643860,3.101522,4.015536
ik,3.803815,3.665109,4.028663,3.802530,4.499704,3.337869,3.856282
sun,3.722750,3.554025,3.996262,4.011587,4.393395,3.250412,3.821405
mean model,3.802794,3.689735,3.986168,3.941100,4.433934,3.346488,3.866703


In [17]:
df_f03_mean_model = df_f03.loc['mean model']
df_f03_mean_context = df_f03.transpose().loc['mean context']

### Faces with $\alpha$ = 0.5

In [18]:
df_f05 = pd.DataFrame(data=f05_scores, columns=model_f05, index=context_f05)
df_f05.columns.name='models'
df_f05.index.name='context'

# Rename columns of the face plot
col = {'aim_face_05':'aim', 'cas_face_05':'cas', 'covsal_face_05':'covsal', 'gbvs_face_05':'gbvs',
       'icf_face_05':'icf', 'ik_10_face_05':'ik_10', 'sun_face_05':'sun', 'ik_face_05':'ik'}
df_f05 = df_f05.rename(index=str, columns=col).transpose()

df_f05

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,3.621147,3.531901,3.765820,3.851181,4.335948,2.762576
cas,4.065310,3.994718,4.179743,4.205287,4.679961,3.898440
covsal,2.953291,2.923755,3.001170,2.827348,3.325123,2.819411
gbvs,4.409638,4.295767,4.594228,4.612659,4.982618,4.198131
icf,3.969019,3.761984,4.304633,4.305627,4.641397,3.072100
ik,3.797096,3.654073,4.028943,3.808641,4.484631,3.307266
sun,3.736457,3.575580,3.997246,4.019056,4.415944,3.259311


#### Get Mean Values across Contexts and Models

In [19]:
df_f05['mean context'] = df_f05.mean(axis=1)
df_f05 = df_f05.transpose()
df_f05['mean model'] = df_f05.mean(axis=1)
df_f05 = df_f05.transpose()
df_f05

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,3.621147,3.531901,3.765820,3.851181,4.335948,2.762576,3.644762
cas,4.065310,3.994718,4.179743,4.205287,4.679961,3.898440,4.170577
covsal,2.953291,2.923755,3.001170,2.827348,3.325123,2.819411,2.975016
gbvs,4.409638,4.295767,4.594228,4.612659,4.982618,4.198131,4.515507
icf,3.969019,3.761984,4.304633,4.305627,4.641397,3.072100,4.009126
ik,3.797096,3.654073,4.028943,3.808641,4.484631,3.307266,3.846775
sun,3.736457,3.575580,3.997246,4.019056,4.415944,3.259311,3.833932
mean model,3.793137,3.676825,3.981683,3.947114,4.409375,3.331033,3.856528


In [20]:
df_f05_mean_model = df_f05.loc['mean model']
df_f05_mean_context = df_f05.transpose().loc['mean context']

### Faces with $\alpha$ = 0.7

In [21]:
df_f07 = pd.DataFrame(data=f07_scores, columns=model_f07, index=context_f07)
df_f07.columns.name='models'
df_f07.index.name='context'

# Rename columns of the face plot
col = {'aim_face_07':'aim', 'cas_face_07':'cas', 'covsal_face_07':'covsal', 'gbvs_face_07':'gbvs',
       'icf_face_07':'icf', 'ik_10_face_07':'ik_10', 'sun_face_07':'sun', 'ik_face_07':'ik'}
df_f07 = df_f07.rename(index=str, columns=col).transpose()

df_f07

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,3.587259,3.487799,3.748489,3.852371,4.218584,2.741446
cas,4.066109,3.996429,4.179063,4.210123,4.667054,3.894861
covsal,2.789149,2.667026,2.987118,2.812137,3.027352,2.669817
gbvs,4.407047,4.291430,4.594469,4.618031,4.974742,4.180200
icf,3.923243,3.688618,4.303583,4.307592,4.564235,2.977367
ik,3.771581,3.613663,4.027574,3.812851,4.471487,3.238392
sun,3.737136,3.576466,3.997592,4.024575,4.417883,3.259335


In [22]:
df_f07['mean context'] = df_f07.mean(axis=1)
df_f07 = df_f07.transpose()
df_f07['mean model'] = df_f07.mean(axis=1)
df_f07 = df_f07.transpose()
df_f07

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,3.587259,3.487799,3.748489,3.852371,4.218584,2.741446,3.605991
cas,4.066109,3.996429,4.179063,4.210123,4.667054,3.894861,4.168940
covsal,2.789149,2.667026,2.987118,2.812137,3.027352,2.669817,2.825433
gbvs,4.407047,4.291430,4.594469,4.618031,4.974742,4.180200,4.510987
icf,3.923243,3.688618,4.303583,4.307592,4.564235,2.977367,3.960773
ik,3.771581,3.613663,4.027574,3.812851,4.471487,3.238392,3.822591
sun,3.737136,3.576466,3.997592,4.024575,4.417883,3.259335,3.835498
mean model,3.754503,3.617347,3.976841,3.948240,4.334477,3.280203,3.818602


In [23]:
df_f07_mean_model = df_f07.loc['mean model']
df_f07_mean_context = df_f07.transpose().loc['mean context']

### Create Mean Matrices

In [24]:
mean_models = pd.DataFrame([df_mean_model, df_f03_mean_model, df_f05_mean_model, df_f07_mean_model], index=[0,0.3,0.5,0.7])

mean_models = mean_models.transpose()
mean_models = mean_models.loc[['all', 'person', 'no person', 'animal', 'electronic', 'sports']]
mean_models

,0.0,0.3,0.5,0.7
context,,,,
all,3.696461,3.802794,3.793137,3.754503
person,3.544559,3.689735,3.676825,3.617347
no person,3.942704,3.986168,3.981683,3.976841
animal,3.804368,3.941100,3.947114,3.948240
electronic,4.327730,4.433934,4.409375,4.334477
sports,3.132214,3.346488,3.331033,3.280203


In [25]:
mean_contexts = pd.DataFrame([df_mean_context, df_f03_mean_context, df_f05_mean_context, df_f07_mean_context], index=[0,0.3,0.5,0.7])

mean_contexts = mean_contexts.transpose()
mean_contexts = mean_contexts.loc[['aim', 'cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun']]
mean_contexts

,0.0,0.3,0.5,0.7
aim,3.626158,3.643184,3.644762,3.605991
cas,4.152310,4.168340,4.170577,4.168940
covsal,3.005601,3.053480,2.975016,2.825433
gbvs,4.498104,4.508696,4.515507,4.510987
icf,4.010667,4.015536,4.009126,3.960773
ik,3.838626,3.856282,3.846775,3.822591
sun,3.791813,3.821405,3.833932,3.835498


## List all the df and save them

In [26]:
df_feaces = pd.DataFrame(data=faces)

In [27]:
os.chdir('../notebooks/data/')

In [28]:
df.to_csv('scores_' + plot_metric + '.csv')
df_feaces.to_csv('scores_faces_' + plot_metric + '.csv')
df_f03.to_csv('scores_f03_' + plot_metric + '.csv')
df_f05.to_csv('scores_f05_' + plot_metric + '.csv')
df_f07.to_csv('scores_f07_' + plot_metric + '.csv')
mean_models.to_csv('scores_mean_models_' + plot_metric + '.csv')
mean_contexts.to_csv('scores_mean_contexts_' + plot_metric + '.csv')